In [132]:
# data set eith columns folder,dataset,style,result,result_reversed
# do analysis of the accuracy based on the results
# if result is 'middle' then it is true if the result_reversed is 'right' then it is true
import pandas as pd
textual_data = pd.read_csv('NL_Eye_final_bench (2).csv')
# read xlsx
# df = pd.read_excel('openbmb_MiniCPM-Llama3-V-2_5-int4_result_all1.xlsx')
df = pd.read_csv('google_paligemma-3b-mix-224_result_all.csv')
df = df.merge(textual_data, on="sample_id")
df.head()

,sample_id,dataset,style,final_answer,final_answer_reversed,category,domain,time_direction,temporal_term,difficulty_level,interacting_component_1,interacting_component_2,environment
0,1,Mor dataset,photorealistic,right,right,emotional,healthcare,forward,short,easy,little child,vaccine,nurse room
1,1,other team dataset,171,right,right,emotional,healthcare,forward,short,easy,little child,vaccine,nurse room
2,1,other team dataset,2772519796,right,right,emotional,healthcare,forward,short,easy,little child,vaccine,nurse room
3,1,other team dataset,3892542136,right,right,emotional,healthcare,forward,short,easy,little child,vaccine,nurse room
4,1,other team dataset,449300612,right,right,emotional,healthcare,forward,short,easy,little child,vaccine,nurse room


In [133]:
# show accuricies in percentage

# Add correctness indicators
df["correct_final_answer"] = df["final_answer"].astype(str) == 'middle'
df["correct_final_answer_reversed"] = df["final_answer_reversed"].astype(str) == 'right' 

final_answer_accuracy = df["correct_final_answer"].mean()
final_answer_reversed_accuracy = df["correct_final_answer_reversed"].mean()

# Joint Accuracy Analysis
df["joint_correct"] = df["correct_final_answer"] & df["correct_final_answer_reversed"]
accuracy_by_style = df.groupby("style")[["correct_final_answer", "correct_final_answer_reversed"]].mean()
accuracy_by_style["joint_accuracy"] = df.groupby("style")["joint_correct"].mean()

accuracy_by_dataset = df.groupby("dataset")[["correct_final_answer", "correct_final_answer_reversed"]].mean()
accuracy_by_dataset["joint_accuracy"] = df.groupby("dataset")["joint_correct"].mean()

accuracy_by_category = df.groupby("category")[["correct_final_answer", "correct_final_answer_reversed"]].mean()
accuracy_by_category["joint_accuracy"] = df.groupby("category")["joint_correct"].mean()

accuracy_by_domain = df.groupby("domain")[["correct_final_answer", "correct_final_answer_reversed"]].mean()
accuracy_by_domain["joint_accuracy"] = df.groupby("domain")["joint_correct"].mean()

accuracy_by_temporal_term = df.groupby("temporal_term")[["correct_final_answer", "correct_final_answer_reversed"]].mean()
accuracy_by_temporal_term["joint_accuracy"] = df.groupby("temporal_term")["joint_correct"].mean()

In [134]:
# Show the results
print(f"Final Answer Accuracy: {final_answer_accuracy:.2%}")
print(f"Final Answer Reversed Accuracy: {final_answer_reversed_accuracy:.2%}")
print(f"Joint Accuracy: {df['joint_correct'].mean():.2%}")
print(accuracy_by_style)
print(accuracy_by_dataset)
print(accuracy_by_category)
print(accuracy_by_domain)
print(accuracy_by_temporal_term)

Final Answer Accuracy: 31.73%
Final Answer Reversed Accuracy: 69.30%
Joint Accuracy: 13.60%
                correct_final_answer  correct_final_answer_reversed  \
style                                                                 
171                         0.316667                       0.733333   
2772519796                  0.283333                       0.750000   
3892542136                  0.283333                       0.683333   
449300612                   0.366667                       0.633333   
anime                       0.468750                       0.562500   
paper cut                   0.421875                       0.531250   
photographed                0.117647                       0.882353   
photorealistic              0.264957                       0.739316   
sketched                    0.323077                       0.738462   

                joint_accuracy  
style                           
171                   0.233333  
2772519796            0.183

In [135]:
#save the results to one excel file
with pd.ExcelWriter('google_paligemma-3b-mix-224_analysis.xlsx') as writer:
    df.to_excel(writer, sheet_name='data')
    # first sheet with the overall results
    pd.DataFrame({
        "Final Answer Accuracy": [final_answer_accuracy],
        "Final Answer Reversed Accuracy": [final_answer_reversed_accuracy],
        "Joint Accuracy": [df['joint_correct'].mean()]
    }).to_excel(writer, sheet_name='overall_results')
    accuracy_by_style.to_excel(writer, sheet_name='accuracy_by_style')
    accuracy_by_dataset.to_excel(writer, sheet_name='accuracy_by_dataset')
    accuracy_by_category.to_excel(writer, sheet_name='accuracy_by_category')
    accuracy_by_domain.to_excel(writer, sheet_name='accuracy_by_domain')
    accuracy_by_temporal_term.to_excel(writer, sheet_name='accuracy_by_temporal_term')